In [2]:
## Imports
import simpleaudio as sa
import numpy as np
from qiskit import *
from qiskit.visualization import plot_bloch_multivector, circuit_drawer
from PIL import ImageTk, Image
import tkinter as tk
import qutip
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)

# pi
pi = np.pi

def letter(phi):
    note_list = ['C','Db','D','Eb','E','F','Gb','G','Ab','A','Bb','B']
    return note_list[int(round((6/pi)*phi)) % 12]

# Function that generates the tones
def sound(circ,note): # Circuit is a quantum circuit and note is the base note

    # Change basenote
    if (note == 'C'):
        f = 440*2**(-9/12)
    if (note == 'C#') or (note == 'Db'):
        f = 440*2**(-8/12)
    if (note == 'D'):
        f = 440*2**(-7/12)
    if (note == 'D#') or (note == 'Eb'):
        f = 440*2**(-6/12)
    if (note == 'E'):
        f = 440*2**(-5/12)
    if (note == 'F'):
        f = 440*2**(-4/12)
    if (note == 'F#') or (note == 'Gb'):
        f = 440*2**(-3/12)
    if (note == 'G'):
        f = 440*2**(-2/12)
    if (note == 'G#') or (note == 'Ab'):
        f = 440*2**(-1/12)
    if (note == 'A'):
        f = 440*2**(0/12)
    if (note == 'A#') or (note == 'Bb'):
        f = 440*2**(1/12)
    if (note == 'B'):
        f = 440*2**(2/12)
    
    # Get statevector
    backend = BasicAer.get_backend('statevector_simulator')
    result = execute(circ, backend).result()
    sv = result.get_statevector()
    
    # Get angles
    theta = np.real(2*np.arccos(sv[0]))
    phi = np.real(np.angle(sv[1]))
    if phi < 0: # Ensure that phi is between 0 and pi
        phi = 2*pi + phi
      
    fs = 44100  # 44100 samples per second
    seconds = 0.5  # Note duration of 2 seconds
    t = np.linspace(0, seconds, seconds * fs, False) # Generate array with seconds*sample_rate steps

    # Create sound
    left =  (np.cos(theta/2))*np.sin(f*t*2*pi) # Left ear
    right = (np.sin(theta/2))*np.sin(2**(phi/(2*np.pi))*f*t*2*pi+pi/2) # Right ear
    
    note = np.dstack((left,right))[0] # Combine into stereo

    # Ensure that highest value is in 16-bit range
    audio = note * (2**15 - 1) / np.max(np.abs(note))
    audio = audio.astype(np.int16) # Convert to 16-bit data
    
    # Playback
    play_obj = sa.play_buffer(audio, 2, 2, fs)
    play_obj.wait_done()

# Initialize quantum circuit
q = QuantumRegister(1, 'q')
circ = QuantumCircuit(q)

# Create tkinter window
root = tk.Tk()
root.title('Mubits')
root.geometry("400x400")

# Get initial quantum Bloch vector
backend = BasicAer.get_backend('statevector_simulator')
result = execute(circ, backend).result()
sv = result.get_statevector()
fig = plot_bloch_multivector(sv)

# Plot the Bloch sphere
canvas = FigureCanvasTkAgg(fig, master=root)
canvas.draw()
canvas.get_tk_widget().place(x = 50,y = 50)

# List of notes
entry_list = ['c','C','d','D','e','f','F','g','G','a','A','b']
note_list = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B','C','C#','D','D#','E','F','F#','G','G#','A','A#','B']

# Heading
head = tk.Text(root)
head.insert(tk.INSERT, 'One Qubit')
head.place(x = 165,y = 10,height = 30,width = 80)

# Initial qubit State
text = tk.Text(root)
text.insert(tk.INSERT, '|q> = 1.0C|L> + 0.0C|R>')
text.place(x = 100,y = 30,height = 30,width = 200)
    
def key_press(event): 

    # Connect keys to quantum gates
    event.char = event.keysym
    if event.char == 'h':
        circ.h(q[0])
    if event.char == '1':
        circ.x(q[0])
    if event.char == '2':
        circ.y(q[0])
    if event.char == '3':
        circ.z(q[0])
    if event.char == 'x':
        circ.rx(pi/6,q[0])
    if event.char == 'y':
        circ.ry(pi/6,q[0])
    if event.char == 'z':
        circ.rz(pi/6,q[0])
    if event.char == 'X':
        circ.rx(-pi/6,q[0])
    if event.char == 'Y':
        circ.ry(-pi/6,q[0])
    if event.char == 'Z':
        circ.rz(-pi/6,q[0])
    
    # Change base note
    index = 0
    for i in range(12):
        if event.char == entry_list[i]:
            index = i
            sound(circ,note_list[i]) 

    # Get current Bloch sphere
    backend = BasicAer.get_backend('statevector_simulator')
    result = execute(circ, backend).result()
    sv = result.get_statevector()
    fig = plot_bloch_multivector(sv)
    
    # Make list of coefficients and angles
    r = []
    phi = []
    for i in range(2):
        r.append(abs(sv[i]))
        p = np.real(np.angle(sv[i]))
        if p < 0:
            p = 2*pi + p
        phi.append(p)
    
    # Plot the Bloch sphere
    canvas = FigureCanvasTkAgg(fig, master=root)
    canvas.draw()
    canvas.get_tk_widget().place(x = 50,y = 50)
    
    # Update quantum state text
    state = '|q> = ' + str(round(r[0],3)) + note_list[index] + '|L> +' + str(round(r[1],3)) + note_list[index+int(round((6/pi)*phi[1]))] + '|R>' 
    text.delete(1.0,tk.END)
    text.insert(tk.INSERT, state)  

# Take in keyboard strikes
root.bind('<Key>', lambda a : key_press(a)) 

# Put tkinter window on top and run mainloop()
root.lift()
root.attributes('-topmost',True)
root.mainloop()